# Geological Interpretor Development

This is a notebook for testing and developping some of the basic code in this package.

## Testing ontology manipulation

The knowledge manipulated in this package is formalised in an ontology,<br>
which is store in a *.owl* file.

It is named **MOGI** for **M**inimal **O**ntology for **G**eological **I**nterpretation

To manipulated this ontology, we use the package **owlready2** available from here: https://owlready2.readthedocs.io

In [ ]:
import owlready2 as owl

In [ ]:
owl.onto_path.append("../ontologies/")
mogi = owl.get_ontology("mogi.owl").load()
mogi

Ontology provides access to its components, e.g.:
* classes
* properties
* individuals
* rules

In [ ]:
print(list(mogi.classes()))
print(list(mogi.properties()))
print(list(mogi.individuals()))
print(list(mogi.rules()))

More specific elements can be searched through simple queries:

In [ ]:
mogi.search(iri = "*Surface")

### Reasoner

Ontologies are even more powerful thansk to their capabilities to use reasoning for infering types, properties, and relationships that were not explicitly stated.
This is usefull for obtaining results implied by the already stated information.

This is achieved by running a *reasoner* on the ontology as follows.

In [ ]:
owl.sync_reasoner(infer_property_values=True)

## Geological Knowledge Manager

**GeologicalKnowledgeManager** may know different instances of **GeologicalKnowledgeFramework**,<br>
for example to allow differenciating scenarios or for allowing customisation of knowledge and its formalisation.

**GeologicalKnowledgeFramework** provides access to concept definitions for providing knowledge.

In [ ]:

import os

class GeologicalKnowledgeManager(object):
    """GeologicalKnowledgeManager is managing one or several GeologicalKnowledgeFramework.
    
    The GeologicalKnowledgeManager is typically a singleton, so there is always one and only one instance of it.
    
    The GeologicalKnowledgeManager may know different instances of GeologicalKnowledgeFramework,
    for example to allow different interpretation scenarios or for allowing user-specific customisation
    of knowledge and its formalisation.
    
    GeologicalKnowledgeFramework are typically ontologies and extensions defined in this package or elsewhere.
    """
    
    def __new__(cls):
        """Method to access (and create if needed) the only allowed instance of this class.
        
        Returns:
        - an instance of GeologicalKnowledgeManager"""
        if not hasattr(cls, 'instance'):
            cls.instance = super(GeologicalKnowledgeManager, cls).__new__(cls)
            cls.initialised= False
        return cls.instance
        
    def __init__(self, default= "mogi", default_source_directory= "../ontologies/", default_source_file= "mogi.owl", default_ontology_backend= "owlready2"):
        """Initializes the GeologicalKnowledgeManager with some default values from configuration.
        
        Parameters:
        - default: specifies the name of the default knowledge framework
        - default_source_directory: specifies the default folder containing of the knowledge framework definitions
        - default_source_file: file contained in the source_directory defining the knowledge framework (e.g., .owl file)
        - default_ontology_backend: specifies the default ontology backend to be used
        """
        if not self.initialised:
            self._initialise(default= default, default_source_directory= default_source_directory, default_source_file= default_source_file, default_ontology_backend= default_ontology_backend)
            
    def _initialise(self, default, default_source_directory, default_source_file, default_ontology_backend):
        """Initializes the GeologicalKnowledgeManager with some default values from configuration.
        
        Parameters:
        - default: specifies the name of the default knowledge framework
        - default_source_directory: specifies the default folder containing of the knowledge framework definitions
        - default_source_file: file contained in the source_directory defining the knowledge framework (e.g., .owl file)
        - default_ontology_backend: specifies the default ontology backend to be used
        """
        self.default= default
        self.default_source_directory= default_source_directory
        self.default_source_file= default_source_file
        self.default_ontology_backend= default_ontology_backend
        
        self.knowledge_framework_dict = {}
        
        self.initialised= True
        
    def reset(self, default= "mogi", default_source_directory= "../ontologies/", default_source_file= "mogi.owl", default_ontology_backend= "owlready2"):
        """Reinitializes the GeologicalKnowledgeManager with some default values from configuration.
        
        Parameters:
        - default: specifies the name of the default knowledge framework
        - default_source_directory: specifies the default folder containing of the knowledge framework definitions
        - default_source_file: file contained in the source_directory defining the knowledge framework (e.g., .owl file)
        - default_ontology_backend: specifies the default ontology backend to be used
        """
        self._initialise(default= default, default_source_directory= default_source_directory, default_source_file= default_source_file, default_ontology_backend= default_ontology_backend)
             
    def load_knowledge_framework(self, name=None, source= None, source_directory= None, backend= None):
        """Gets and initilises the ontology from the specified source.
        
        Parameters:
        - name: the name to be given to the knowledge framework. If None (default) the file name will be used.
        - source: filename to the ontology source. If None(default) the default ontology is used.
        - source_directory: where the system should look for ontology definition files. If None, the `GeologicalKnowledgeFramework` will decide.
        - backend: the ontology backend to be used. If None, the `GeologicalKnowledgeFramework` will decide."""
        source = source if source is not None else self.default_source_file
        name = name if name is not None else os.path.basename(source).split(os.path.extsep)[0]
        self.knowledge_framework_dict[name] = GeologicalKnowledgeFramework(name= name, source= source, source_directory= source_directory, backend= backend)
    
    def get_knowledge_framework(self,name= "default"):
        """Accessor to knowledge frameworks."""
        name = self.default if name == "default" else name
        assert len(self.knowledge_framework_dict) > 0, "No ontology has been loaded yet. Please use GeologicalKnowledgeManager().load_knowledge_framework() first"
        assert name in self.knowledge_framework_dict.keys(), "The specified ontology hasn't been loaded: "+name+\
            "\navailable ontology names are: "+"\n".join(self.knowledge_framework_dict.keys())
        return self.knowledge_framework_dict[name]
    
class GeologicalKnowledgeFramework(object):
    """A GeologicalKnowledgeFramework holds the definition of concepts and relationships describing knowledge.
    
    This is typically an overlay around a formal ontology definition, which also brings additional capabilities,
    such as algorithms and factories to achieve specific tasks and create objects."""
    
    def __init__(self, name, source, source_directory= None, backend= None):
        """Initialise a KnowledgeFramework form a given ontology file (source).
        
        Parameters:
        - name: should be the name under which this KnowledgeFramework is known in the manager
        - source: the source file for the ontology definition
        - source_directory: the directory where the source files for the ontology definition are looked for.
        If None (default) the default path provided by the `KnowledgeManager` is used.
        - backend: the ontology backend to be used for this knwoledge framework.
        If None (default) the default ontology backend provided by the `KnowledgeManager` is used."""
        self.name= name
        
        self.__source_directory= None
        self.init_source_directory(source_directory)
        self.initialise_ontology_backend(backend)
        
        self.load_ontology(source)
    
    def init_source_directory(self, source_directory):
        """Initialises the folder where source files are searched.
        
        Parameters:
        - source_directory: if None, the previous value is used if it wasn't None, else the `GeologicalKnowledgeManager`default is used."""
        if source_directory is not None:
            self.__source_directory= source_directory
        elif self.__source_directory is None:
            self.__source_directory= GeologicalKnowledgeManager().default_source_directory
    
    def initialise_ontology_backend(self, backend_name:str= None):
        """Initializes the ontology package used as a backend to access ontologies.
        
        This will:
        - try to import the backend as onto
        - set the default path for ontologies"""
                
        self.__ontology_backend = None
        backend_name= GeologicalKnowledgeManager().default_ontology_backend if backend_name is None else backend_name
        if backend_name == "owlready2":
            try:
                import owlready2 as owl2 
                self.__ontology_backend = owl2
                if self.__source_directory not in self.__ontology_backend.onto_path:
                    self.__ontology_backend.onto_path.append(self.__source_directory)
            except ImportError:
                raise ImportError("Your are trying to use Owlready2 as a backend for ontology management, but it doesn't appear to be installed."\
                "This is either because OwlReady2 is given as default option or because you asked for it."\
                "Please install the OwlReady2 package from https://owlready2.readthedocs.io"\
                "or give another backend through GeologicalKnowledgeManager().initialise_ontology_backend()")
                
            # also test if java is correctly installed & accessible, as it is used by owlready2 for reasoning
            try:
                os.system("java -version")
            except:
                raise ImportError("Java doesn't appear to be installed properly as the command `java -version` returned an error."\
                    "This error occured while loading owlready2 package as an ontology backend, because java is used for the reasoning engine.")
        else:
            raise Exception("The specified backed for ontology is not supported: "+backend_name)
          
        
    def load_ontology(self, source):
        """Loads the ontology specified by source.
        
        Parameters:
        - source: the source file for the ontology definition
        - source_directory: the directory where the source files for the ontology definition are looked for.
        If None (default) the default path provided by the `KnowledgeManager` is used."""
        self.__source= source
        try:
            self.__onto = self.__ontology_backend.get_ontology(self.__source).load()
        except Exception as err:
            raise Exception("Unexpected exception received while loading ontology:\n - source: {}\n - onto_path: {}".format(self.__source, self.__ontology_backend.onto_path))
        
    def __call__(self):
        return self.__onto
        
    def get_ontology_backend(self):
        """Gets the ontology backend"""
        assert self.__ontology_backend is not None, "Trying to access the ontology backend without initialising it."
        return self.__ontology_backend
    
    def search(self, name= None, type= None, qualities= None, prepend_star=True) -> list:
        """Search function to interface the serach capabilities of the internal ontology
        
        Parameters:
        - name: the name of the search object (you can use * to replace any set of characters and ? to replace any single character)
        Note: if `prepend_star` a * is always prepended to allows the search to work because of the internal prefix names
        - type: the type of the searched observations as defined by the internal ontology
        - qualities: qualities to filter the observations.
        If `qualities` is a :
         * `str`: a single quality will be searched for with any value ("*"),
         * `list`: a list of qualities will be searched for with any values ("*")
         * `dict`: a list of qualities defined by the keys and with the associated values will be searched for"""
        if name is None:
            name = "*"
        elif prepend_star:
            name = "*"+name
        
        if isinstance(qualities,list):
            kargs = {quality_i: "*" for quality_i in qualities} 
        elif isinstance(qualities,str):
            kargs = {qualities: "*"}
        elif isinstance(qualities,dict):
            kargs = qualities
        else:
            kargs = {}
            assert (qualities is None) or isinstance(qualities,dict), "qualities should be given as either None, a str, a list, or a dict"
        if type is not None: kargs["type"] = type
        return self.__onto.search(iri= name, **kargs)
    
    def remove_entity(self, entity):
        self.__ontology_backend.destroy_entity(entity)
        
    def remove_entities(self, entities):
        for entity in entities:
            self.remove_entity(entity)
        
    def show_entity_qualities(self,entity):
        print("Entity:",entity)
        for prop in entity.get_properties():
            print("{}:{}".format(prop.name, prop[entity]))
            
    def has_quality(self, object, quality):
        return getattr(self.__onto, quality) in object.get_properties()
    
    def has_qualities(self, object, qualities):
        qualities = np.array(qualities).ravel()
        return np.all([self.has_quality(object, quality_i) for quality_i in qualities])
            
    def sync_reasoner(self, **kargs):
        """Synchronise the reasoner.
        
        Parameters:
        - **kargs:
        |-infer_property_values"""
        self.__ontology_backend.sync_reasoner(**kargs)
        
    def __str__(self):
        """Describe the knowledge framework"""
        desc = ["Geological Knowledge Framework:"]
        desc += [" |- Name: {}".format(self.name)]
        desc += [" |- Backend: {}".format(self.__ontology_backend)]
        desc += [" |- Source: {}".format(self.__source)]
        desc += [" |- Ontology: {}".format(self.__onto)]
        return "\n".join(desc)
    

In [ ]:
GeologicalKnowledgeManager().load_knowledge_framework()
print(GeologicalKnowledgeManager().get_knowledge_framework())

In our approach, geological datasets will be progressively interpreted in terms of structural objects,<br>
based on a formal definition of concepts own by a **GeologicalKnowledgeManager**.<br>


In [ ]:
mogi = GeologicalKnowledgeManager().get_knowledge_framework()
mogi.name

In [ ]:
mogi().classes()

In [ ]:
mogi().search(type= mogi().Ponctual_Observation)

In [ ]:
mogi.search(type= mogi().Ponctual_Observation, qualities= ["dip","occurrence"])

In [ ]:
mogi.search(type= mogi().Ponctual_Observation, qualities= {"dip":45})

In [ ]:
mogi.search(qualities= {"dip":45})

In [ ]:
mogi.search()

In [ ]:
mogi.search(name="*D1?")

In [ ]:
mogi.search(name="D1")

## Representation Spaces

We distinguish two kind of operations here:
* representation
* visualisation

A representation is a formal description of how something appears in a given representation space, but it doesn't have to be visualised.<br>
A visualisation takes care of the rendering of a representation with a given support (image, screen).

Representation should also be made a bit more abstract.<br>
1. There is a variety of object that can be rendered in a representation space (typically, different kinds of a dataset components)
2. Several kinds of representation spaces could be envisionned (e.g., spatial 1D,2D,3D, or temporal, or just an abstract text)

In [ ]:

import numpy as np

class RepresentationSpace(object):
    """A general framework for Representating geological objects"""
    
class TemporalRepresentationSpace(RepresentationSpace):
    """A `RepresentationSpace` representing temporal apsects of represented objects."""
    
class PhysicalRepresentationSpace(RepresentationSpace):
    """A type of `RepresentationSpace` representing physical aspects of the represented objects."""
    
    __default_coordinate_labels = ["X","Y","Z"]
    
    def __init__(self, dimension: int=None, coordinate_label: str|list= None, dataset= None, **kargs):
        """Initialisation of the representation space.
        
        Parameters:
        - dimension (int): specify the number of dimensions of the representation space, typically 1D, 2D, or 3D (i.e., 1, 2, or 3),
        NB: larger dimension spaces are not supported. At least either the `dimension` parameter or `coordinate_label` parameter should be given.
        - coordinate_label(str|list(str)): gives the label(s) of the coordinates. If given, the number of dimensions is deduced from the size of the list
        and `dimensions`is ignored, otherwise, the labels are taken from the `__default_coordinate_labels` based on the number of `dimension`s. 
        At least either the `dimension` parameter or `coordinate_label` parameter should be given.
        - dataset: a Dataset object containing the data to be attached to this representation space.
        Note that the RepresentationSpace can be created first and then updated automatically when creating the dataset attached to this space.
        - **kargs:
            - use_extension: if True, uses the extension of the dataset, else keeps the current ones
            - padding: if use_extension is True, the given paddign will be used to keep a space around the dataset
        """
        assert not (coordinate_label is None and dimension is None), "At least one of the parameters should be specified"
        if coordinate_label is None:
            assert isinstance(dimension, int),"dimension parameter must be an integer"
            assert dimension in [1,2,3], "The specified number of dimensions ({:d}) is not supported, should be 1, 2 or 3.".format(dimension)
            self.dimension= dimension
            self.coordinate_labels= PhysicalRepresentationSpace.__default_coordinate_labels[:self.dimension]
        elif isinstance(coordinate_label,str):
            self.dimension= 1
            self.coordinate_labels=  [coordinate_label]
        elif isinstance(coordinate_label, list):
            self.dimension= len(coordinate_label)
            self.coordinate_labels= coordinate_label
        else:
            raise("Unsupported initialisation of representation space: dimension({}) and coordinate_label ({}).\n At least one of the parameters shoudl be specified.".format(dimension, coordinate_label))

        self.__default_padding= 0.1
        self.__datasets= []
        self.set_extension()
        self.attach_dataset(dataset,**kargs)
        
    def attach_dataset(self, dataset, use_extension= True, padding= None, **kargs):
        """Attach a dataset to the representation space
        
        Parameters:
        - dataset: a Dataset object to be attached
        - use_extension: if True, uses the extension of the dataset, else keeps the current ones
        - padding: if use_extension is True, the given paddign will be used to keep a space around the dataset"""
        if dataset == None: return
        if dataset in self.__datasets: return
        self.__datasets += [dataset]
        
        if use_extension:
            self.set_extension_from_data(padding= padding)
        
        dataset.setup_representation_space(physical_space= self)
    
    def get_datasets(self):
        """dataset getter"""
        return self.__datasets
        
    def set_extension(self, extension:list= None):
        """Setter for the extension (min,max) of the representation space
        
        Parameters:
        - extension: a list containing a pair of min and max value for each dimension of the space.
        If None, the default will be set, i.e., [[0,1]] * dimension"""
        if extension is None: 
            self.extension = [[0,1]] * self.dimension
            return
        extension= np.array(extension)
        assert extension.shape[0] == self.dimension, "The specified extension ({}) do not match the space dimensions ({})".format(extension, self.dimension)
        assert extension.shape[1] == 2, "The specified extension should provide both lower andupper bounds for each dimension, given: {}".format(extension)
        self.extension= extension
        
    def set_extension_from_data(self, padding= None):
        """Sets the extension of the space from the attached dataset
        
        If no dataset is attached yet, then default extension are used instead (min:0,max:1).
        
        Parameters:
        - padding: a space that is left around the dataset, either a value compatible with the coordinates, or a list of values of same dimensions.
        If None, by default the padding is 5% of the dataset range.
        """
        non_empty_dataset = [data_i for data_i in self.__datasets if data_i.extension is not None]
        if len(non_empty_dataset) == 0:
            self.set_extension()
            return
        
        if padding is None:
            padding= self.__default_padding
        else:
            try: # check if padding as a dimension
                len(padding)
            # if not, then use it a a scaling 
            except TypeError: #just checking it is a number
                assert type(padding) == int or type(padding) == float, "padding should be given as a number (int or float), here: "+type(padding)
                # keep the padding as is in this case
            else:# else check its dimensions are ok
                assert len(padding) == self.dimension, "the dimensions of the specified padding (len({})->) should match the space dimension ({})".format(padding, len(padding), self.dimension)
                padding= np.array(padding)
                
        extension = non_empty_dataset[0].extension
        for data_i in non_empty_dataset[1:]:
            for dim_i in self.dimension:
                extension[dim_i,0] = min(extension[dim_i,0], data_i.extension[dim_i])
                extension[dim_i,1] = max(extension[dim_i,1], data_i.extension[dim_i])
        """:todo: use projected coordinates instead of source coordinates, might fail if 3D data projected on a map"""
                
        # in any cases, except when padding and data is None
        self.center= np.mean(extension, axis= 1)
        diff= extension.T - self.center
        diff= diff.T
        
        # padding is multiplied by the width along each axis and added to the extension
        # check if each dimension is shrinked (ie. min == max -> diff == 0), takes the average extension of the other dimensions,
        # unless all are shrinked, in which case [[-1,1]] * self.dimension is set
        if np.all(diff == 0):
            diff = np.array([[-1,1]] * self.dimension)
        else:
            zero_diff = np.where(~diff.any(axis=1))
            non_zero_diff = np.where(diff.any(axis=1))
            mean_diff = np.mean(diff[non_zero_diff], axis=0)
            diff[zero_diff] = mean_diff
        
        extension= np.repeat([self.center],2,axis=0).T + (1+2*padding)*diff 
        
        self.set_extension(extension)
        
    def filter_qualities(self,**qualities):
        """removes the named arguments corresponding to this space coordinates from qualities.
        
        Return:
        - a copy with the passed parameters except for the ones correpsonding to the space coordinates"""
        return {key:val for key, val in qualities.items() if key not in self.coordinate_labels}
    
    def prepare_coordinate_qualities(self, **qualities):
        """transforms the coordinates passed in qualities into an appropriate format
        
        Return:
        - a dict with quality keys and values for setting coordinates"""
        coordinates = {key:val for key, val in qualities.items() if key in self.coordinate_labels}
        coord_qualities = {}
        for i,key in enumerate(self.coordinate_labels):
            if key in coordinates:
                coord_qualities["coord{}".format(i+1)] = coordinates[key]
                coord_qualities["coord{}_label".format(i+1)] = [key]
        return coord_qualities
    
    def label_map(self, object):
        """creates of mapping of coordinate labels"""
        coord_label_params = ["coord{}_label".format(i) for i in range(1,4) if hasattr(object, "coord{}_label".format(i))]
        label_map = {getattr(object,param)[0]:param for param in coord_label_params if getattr(object,param)[0] in self.coordinate_labels}
        return label_map
        
    def set_object_coordinates(self, object, **kargs):
        """Sets the coordinates corresponding to this space into the given object
        
        Parameters:
        - object: the object whose coordinates needs to be set
        - kargs: keyword argugments corresponding to the name and values of the coordinates.
        They must match this space coordinate names, extra names will be ignored"""
        
        # check if the object has coord{i}_label set, else set it
        if mogi.has_qualities(object, ["coord{}_label".format(i) for i in range(1,len(self.coordinate_labels)+1)]):
            label_map = self.label_map(object)
            coordinate_values = {label_map[key].split("_")[0]:kargs[key] for key in self.coordinate_labels if key in kargs}
        else:
            coordinate_values = self.prepare_coordinate_qualities(**kargs)
        for param_name, val in coordinate_values.items():
            setattr(object, param_name, val)
            
    def get_object_coordinates(self, object):
        """Gets the coordinates corresponding to this space from the given object"""
        label_map = self.label_map(object)
        coord = np.full_like(self.coordinate_labels, np.nan, dtype= float)
        for i, key in enumerate(self.coordinate_labels):
            if key in label_map:
                coord_param = label_map[key].split("_")[0]
                coord[i] = getattr(object,coord_param)[0]
        return coord
    
    def __str__(self):
        """Description of the physical space parameters and data
        """
        desc= ["Representation space of type: {:s}".format(type(self).__name__)]
        desc+= ["- Number of dimension(s): {}".format(self.dimension)]
        desc+= ["- Coordinate label(s): {}".format(self.coordinate_labels)]
        desc+= ["- Space extension:"]
        for dim_i, lim_i in zip(self.coordinate_labels,self.extension):
            desc+= [" |- Coord {}: {}".format(dim_i, lim_i)]  
        return "\n".join(desc)

In [ ]:
space= PhysicalRepresentationSpace(2)

space.set_extension_from_data(padding= None)

print(space)


In [ ]:
PhysicalRepresentationSpace(coordinate_label= "depth")

In [ ]:
try: PhysicalRepresentationSpace()
except Exception as e:
    print(e)

In [ ]:
space= PhysicalRepresentationSpace(coordinate_label=["X","Y"])
print(space)

In [ ]:
space= PhysicalRepresentationSpace(coordinate_label=["X","Z"])
print(space)

## Dataset

Todo:
* visualisation of given data types

In [ ]:
import logging

class GeologicalDataset(object):
    """A GeologicalDataset gathers information about geological data to be interpreted.
    
    This class is a hybrid ontology&python class. It is providing pythonic algorithm and high level interface,
    while the data is actually stored in an ontology.
    """
    
    def __init__(self, physical_space= None, time_space= None, representation_spaces= None, knowledge_framework= None):
        """Initialises a `GeologicalDataset`
        
        Parameters:
        - physical_space: a `PhysicalRepresentationSpace`,  which defines the spatial coordinates of this dataset
        - time_space: a `TemporalRepresentationSpace`,  which defines the time coordinates of this dataset
        - representation_spaces: list of `Representationspace`s to which the dataset must be attached
        Note: datasets can be created without representation space and attached later on by using `RepresentationSpace.attach_dataset`
        or `GeologicalDataset.setup_representation_space`.
        Alternativelly, a single `PhysicalRepresentationspace` and or `TemporalRepresentationspace` can be given here if `physical_space` and `time_space` are None.
        - default_representation_space: the main RepresentationSpace to which this dataset is attached.
        If None and representation_spaces are provided, then the first one will be taken.
        If the default one is not initially in the full list, then it is added to it.
        - ontology: the name of the ontology to be used for storing the data.
        If None, the default will be taken from `the GeologicalKnowledgeManager`.
        
        Internals: this method initialises several internal attributes:
        - extension: represents the extension of the dataset in the attached representation space
        (i.e., the default on if this dataset is represented in several representation spaces
        - representation_spaces: the dataset can be attached to and represented into several representation spaces,
        `physical_space` and `time_space` are included into this list.
        """
        
        self.knowledge_framework= knowledge_framework if knowledge_framework is not None else GeologicalKnowledgeManager().get_knowledge_framework()
        
        self.extension = None
        
        # setup representation spaces
        self.representation_spaces= set()
        if physical_space is None:
            # try to initialise the physical space with first existing data
            observations = self.get_observations()
            if len(observations) > 0:
                coord_labels = [getattr(observations[0],"coord{}_label".format(i)) for i in range(1,4)]
                coord_labels = [label[0] for label in coord_labels if len(label)>0]
                physical_space = PhysicalRepresentationSpace(coordinate_label=coord_labels)
        self.setup_representation_space(physical_space, time_space, representation_spaces)
        
        # register the datast in the listed representation spaces
        for space_i in self.representation_spaces:
            space_i.attach_dataset(self)
        
        # initialize extension of the dataset
        self.update_extension()
            
    def __dell__(self):
        self.remove_all_observations()

    def __setup_space(self, space, space_type):
        """Check if space of given type is in list or parameter and return the appropriate value.
        
        Take the given physical/time space, or if None use the first one in the list, and if none just leave None.
        Adds the space to the `self.representation_spaces` set."""
        if space is not None: 
            self.representation_spaces.add(space)
            return space
        if len(self.representation_spaces) == 0: return None
        space_list= [space_i for space_i in self.representation_spaces if isinstance(space_i, space_type)]
        return space_list[0] if len(space_list) > 0 else None
    
    def update_extension(self, update_representation_spaces= True):
        """Sets the extension of the dataset in the physical space from existing observations
        
        If there is no observation, `self.extension` is set to None"""
        observations = self.get_observations()
        if (len(observations) == 0) or (self.physical_space is None):
            self.extension = None 
            return
            
        di = observations[0]
        coord = self.physical_space.get_object_coordinates(di)
        self.extension = np.repeat([coord],2,axis=0).T
        for di in observations[1:]:
            coord = self.physical_space.get_object_coordinates(di)
            for j, val in enumerate(coord):
                self.extension[j,0] = min(self.extension[j,0], val)
                self.extension[j,1] = max(self.extension[j,1], val)
                
        if update_representation_spaces:
            if self.physical_space is not None:
                self.physical_space.set_extension_from_data()
        
    def setup_representation_space(self, physical_space= None, time_space= None, representation_spaces= None):
        """Setup the representation space list and default
        
        Parameters:
        - physical_space: a `PhysicalRepresentationSpace`,  which defines the spatial coordinates of this dataset
        - time_space: a `TemporalRepresentationSpace`,  which defines the time coordinates of this dataset
        - representation_spaces: list of `Representationspace`s to which the dataset must be attached
        Note: datasets can be created without representation space and attached later on by using `RepresentationSpace.attach_dataset`
        or `GeologicalDataset.setup_representation_space`.
        Alternativelly, a single `PhysicalRepresentationspace` and or `TemporalRepresentationspace` can be given here if `physical_space` and `time_space` are None.
        """
        if representation_spaces is not None: self.representation_spaces = self.representation_spaces.union(representation_spaces)
        self.physical_space = self.__setup_space(physical_space, PhysicalRepresentationSpace)
        self.time_space = self.__setup_space(time_space, TemporalRepresentationSpace)
        
        for space in self.representation_spaces:
            space.attach_dataset(self)
        
    def get_observations(self, observation_type= None, qualities= None, name= None):
        """Accessor to the observations stored in the internal ontology
        
        Parameters:
        - observation_type: the type of the searched observations as defined by the internal ontology
        - qualities: qualities to filter the observations (c.f. `KnowledgeFramework.search`)
        - name: name of the serached observation (c.f. `KnowledgeFramework.search`)"""
        observation_type = observation_type if observation_type is not None else self.knowledge_framework().Ponctual_Observation
        return self.knowledge_framework.search(type= observation_type, qualities= qualities, name= name)
    
    def get_occurrence_observations(self):
        """helper method to access occurrence data, i.e., those having a occurrence quality
        
        :todo: for now the occurrence quality doesn't exist so all the observations are occurrence by default"""
        return self.get_observations(qualities= "occurrence")
    
    def get_orientation_observations(self):
        return self.get_observations(qualities= "dip")
    
    def remove_observation(self, observations, update_extension= True):
        """Removes the given observations stored in this dataset and internal ontology
        
        Parameters:
        - observations: an iterable containing objects of the internal ontology.
        Note that you can use the `search`method to get such a list
        - update_extension: if True (default) the extension will be updated"""
        self.knowledge_framework.remove_entities(observations)
        if update_extension: self.update_extension()
            
    def remove_observation_by_name(self, name:str, update_extension= True):
        """Removes the given observations stored in this dataset and internal ontology
        
        Parameters:
        - name: the name of the observation to be removed
        - update_extension: if True (default) the extension will be updated"""
        self.remove_observation(self.get_observations(name= name), update_extension)
        
    def remove_all_observations(self, update_extension= True):
        """Removes all the observations stored in this dataset and internal ontology
        
        Parameters:
        - update_extension: if True (default) the extension will be updated.
        Note: the update is performed only once at the end."""
        self.remove_observation(self.get_observations(), update_extension= False)
        if update_extension: self.update_extension()
        
    def add_observation(self, name: str, update_extension= True, **kargs):
        """creates a new observation and adds it to te internal ontology
        
        Parameters:
        - name: the name of the observation (similar to an observation id)
        - update_extension: if True (default) the extension will be updated.
        - **kargs:
          |- any argument whose name corresponds to the `physical_space`coordinate labels or other properties.
          |   Note: all the coordinates must be specified
          |   Also Note: the arguments with None value are removed from the dict"""
        if name == "":
            logging.warning("Trying to create an observation with a blank name, passing None instead.")
            name = None
        if not isinstance(name,str):
            logging.warning("Trying to create an observation with a name that is not a string, passing None instead.")
            name = None
        assert self.physical_space is not None, "Trying to add observation while physical space is not set. Please setup_representation_space first"
        
        qualities = {key:[val] for key, val in kargs.items() if val is not None}
        non_coordinate_qualities = self.physical_space.filter_qualities(**qualities)
        
        new_observation = self.knowledge_framework().Ponctual_Observation(name= name, **non_coordinate_qualities)
        self.physical_space.set_object_coordinates(new_observation,**qualities)
        if update_extension: self.update_extension()
        
    def add_occurrence_observation(self, name: str, observed_object:str, occurrence= True, update_extension= True, **kargs):
        """creates a new occurrence observation and adds it to te internal ontology
        
        Parameters:
        - name: the name of the observation (similar to an observation id)
        - observed_object: the name of the observed object
        - occurrence: True (default) if the object was observed here, False if it was observed that it is not there.
        Note that this is different from not having observed that it is here, in which case there should not be an observation.
        - update_extension: if True (default) the extension will be updated.
        - **kargs:
          |- any argument whose name corresponds to the `physical_space`coordinate labels or other properties.
          |   Note: all the coordinates must be specified"""
        if observed_object is not None: kargs["geology"]= observed_object
        if occurrence is not None: kargs["occurrence"]= occurrence
        self.add_observation(name, update_extension= update_extension, **kargs)
    
    def add_orientation_observation(self, name: str, observed_object:str, dip, dip_dir, occurrence= True, update_extension= True, **kargs):
        """creates a new orientation observation and adds it to te internal ontology
        
        Parameters:
        - name: the name of the observation (similar to an observation id)
        - observed_object: the name of the observed object
        - dip: the value of the measured dip (in degrees, 0-90)
        - dip_dir: the value of the dip direction (in degrees, 0-360, from North towards the East)
        - occurrence: True (default) if the object was observed here.
        This is the default behaviour because if the measurement was made here, we assume that the object actually existed
        so this is in itself a proof ox occurrence. However, one might want to record the orientation without specifically attaching any
        observation of occurrence, in which case None should be given for occurrence and the quality won't be set.
        False, would not make much sense as it would imply that the orientation was measured but we observed that the object wasn't there.
        - update_extension: if True (default) the extension will be updated.
        - **kargs:
          |- any argument whose name corresponds to the `physical_space`coordinate labels or other properties.
          |   Note: all the coordinates must be specified"""
        if occurrence is not None: kargs["occurrence"]= occurrence
        if occurrence == False: logging.warning("occurrence parameter was set to False while adding an orientation observation."\
            "This is weird because it would imply the measure was taken but the rock couldn't be observed."\
            "Did you intend to avoid recording the occurrence, in which case you should prefer None isntead of False.")
        
        if observed_object is not None: kargs["geology"]= observed_object
        if (dip is not None) and (dip_dir is not None):
            kargs["dip"]= dip
            kargs["dip_dir"]= dip_dir
        self.add_observation(name, update_extension= update_extension, **kargs)
        
    def __len__(self):
        return len(self.get_observations())
         
    def __str__(self):
        """Description of the dataset
        """
        desc= ["A dataset of type: {:s}".format(type(self).__name__)]
        n = len(self)
        if n == 0:
            desc+= ["- The dataset is empty"]
        else:
            desc+= ["- Size of dataset: {:d}".format(n)]
            desc+= ["- Types of data:"]
            desc+= [" |- occurrence:\t{} entries".format(len(self.get_occurrence_observations()))]
            desc+= [" |- orientation:\t{} entries".format(len(self.get_orientation_observations()))]
        if self.extension is None:
            desc+= ["- Extension: "+str(self.extension)]
        else:
            desc+= ["- Extension:"]
            labels = self.physical_space.coordinate_labels if self.physical_space is not None else ["Coord_{}".format(i) for i in range(3)][:len(self.extension)]
            for dim_i, lim_i in zip(labels,self.extension):
                desc+= [" |- Coord {}: [{}, {}]".format(dim_i, *lim_i)]  
        desc+= ["- Number of spaces this dataset is attached to: {:d}".format(len(self.representation_spaces))]
        if self.physical_space is None:
            desc+= [" |- Physical space: None"]
        else:
            desc+= [" |- Physical space:"+"\n | |".join(self.physical_space.__str__().split("\n"))]
        if self.time_space is None:
            desc+= [" |- Temporal space: None"]
        else:
            desc+= [" |- Temporal space:"+"\n | |".join(self.time_space.__str__().split("\n"))]
        return "\n".join(desc) 
    
    def head(self, n:int= 5):
        """Returns the `n`first data in the dataset"""
        return self.to_dataframe(max_rows=n)
    
    def info(self):
        """Returns a description of the dataset"""
        return self.__str__()
    
    def to_dataframe(self, max_rows:int= None):
        """Creates a `pandas.DataFrame` showing the data in this dataset
        
        Parameters:
        - max_rows: limits the number of rows in the output, unless None is given (default).
        Note: interanlly, all the observations are still recovered from the internal ontology, 
        but only the `max_rows`first ones are show for consision.
        """
        columns = ["name"]
        if self.physical_space is not None:
            columns += self.physical_space.coordinate_labels
        if self.time_space is not None:
            columns += self.time_space.coordinate_labels
        columns += ["dip_dir","dip",'geology', 'occurrence']
        output_frame = pd.DataFrame(columns= columns)
        output_frame.set_index("name",inplace=True)
        
        observations = self.get_observations() 
        observations = observations if max_rows is None else observations[:max_rows]
        for di in observations:
            for prop in di.get_properties():
                if prop.name == '': continue
                if "coord" in prop.name: continue
                output_frame.loc[di.name,prop.name] = prop[di][0]
                
            if self.physical_space is not None:
                for label, val in zip(self.physical_space.coordinate_labels, self.physical_space.get_object_coordinates(di)):
                    output_frame.loc[di.name,label] = val
            if self.time_space is not None:
                for label, val in zip(self.time_space.coordinate_labels, self.time_space.get_object_times(di)):
                    output_frame.loc[di.name,label] = val
        coordinate_types = {label:float for label in self.physical_space.coordinate_labels} if self.physical_space is not None else {}
        time_types = {label:float for label in self.physical_space.coordinate_labels} if self.time_space is not None else {}
        other_types = {'dip_dir':float, 'dip':float, 'geology':str, "occurrence": bool}
        output_frame = output_frame.astype({**coordinate_types, **other_types})
        return output_frame
    
def load_dataset_from_csv(source:str, dataset:GeologicalDataset = None, coordinate_labels = ["X","Y","Z"], **kargs) ->GeologicalDataset:
    """Loads a dataset from a csv file
    
    Parameters:
    - source(str): the source file from which the data should be loaded
    - dataset: the `GeologicalDataset` in which the data will be loaded. If None, the dataset will be created.
    - **arkgs: passed to pandas.read_csv
    
    Return:
    - the `GeologicalDataset` with the newly loaded data (a new `GeologicalDataset` is created if needed)."""
    try:
        dataframe = pd.read_csv(source, **filter_kargs(pd.read_csv,**kargs))
    except Exception as e:
        raise( Exception("This error occurred while loading a dataset from: {}\nAdditional arguments were given: {}".format(source,
                            ",".join(["{}:{}".format(key,val) for key, val in kargs.items()]))))
        
    coordinate_labels = [label for label in coordinate_labels if label in dataframe.columns]
    if len(coordinate_labels) == 0:
        logging.warning("There isn't any coordinate column in the loaded dataset.\nCheck the output and consider changing the separator with sep keyword or cahnge 'coordinate_label' parameter.")
    return load_dataset_from_dataframe(dataframe, dataset, coordinate_labels= coordinate_labels, **filter_kargs(load_dataset_from_dataframe,**kargs) )

def load_dataset_from_dataframe(dataframe, dataset:GeologicalDataset = None, coordinate_labels = ["X","Y","Z"], labels= None, index= None, dtypes= None, clear_existing_data= True):
    """Loads a dataset from a `pandas.DataFrame`
    
    Parameters:
    - dataframe(`pandas.DataFrame`): the source dataframe from which the data should be loaded
    - dataset: the `GeologicalDataset` in which the data will be loaded. If None, the dataset will be created.
    - coordinate_labels: the labels to be used as coordinates of the physical space
    - labels: a dict to relabel the dataframe columns prior to loading in the dataset.
    This is usefull for example when the coordinates in the source aren't labelled the same as in the internal ontology.
    The format is {"old_label":"new_label", ...}.
    - index: the label of the column (in original DataFrame, i.e., before renaming), which is to be used as index
    - dtypes: a dict containing a mapping between column name and type
    - clear_existing_data: if set (default), any oservation stored in the internal ontology is removed prior to loading the new dataset 
    
    Return:
    - the `GeologicalDataset` with the newly loaded data (a new `GeologicalDataset` is created if needed)."""
    if dataset is None:
        physical_space = PhysicalRepresentationSpace(coordinate_label= coordinate_labels)
        dataset = GeologicalDataset(physical_space = physical_space)
    else:
        if dataset.physical_space is None or clear_existing_data:
            physical_space = PhysicalRepresentationSpace(coordinate_label= coordinate_labels)
            dataset.setup_representation_space(physical_space = physical_space)
        else:
            if dataset.physical_space.coordinate_labels != coordinate_labels:
                raise MalissiaBaseError("Trying to add data into an existing dataset with different coordiante labels")
                
    if clear_existing_data:
        dataset.remove_all_observations()

    # declare default dtypes here, in case they should be relabelled
    default_coord_types = {key:float for key in dataset.physical_space.coordinate_labels}
    other_types = {'dip_dir':float, 'dip':float, 'geology':str, 'observed_object':str, "occurrence":bool}
    dtypes= dtypes if dtypes is not None else {**default_coord_types, **other_types}
    assert isinstance(dtypes, dict), "dtypes for type management should be given as a dict"
    
    # relabelling
    if labels is not None:
        dataframe = dataframe.rename(columns= labels)
        index= labels[index] if index in labels else index
        dtypes= {labels[key] if key in labels else key: value for key, value in dtypes.items()}
    
    # setting the index
    if index is not None:
        # if already set, reset it
        if type(dataframe.index) != pd.core.indexes.base.Index:
            dataframe = dataframe.reset_index()
        # then set the index
        try:
            dataframe = dataframe.set_index(index)
        except Exception as e:
            raise( Exception("Error while setting the index of the loaded dataset:\nThis is how the dataframe looks like:\n"+dataframe.head().to_string()))
    
    for extra_type_i in dtypes.keys() - set(dataframe.columns):
        del dtypes[extra_type_i]
    dataframe = dataframe.astype(dtypes)
        
    for name_i, values_i in dataframe.iterrows():
        # drop nan values and filter None objects
        values_i = {key:val for key, val in values_i.dropna().items() if val is not None}
        dataset.add_observation(name_i, **values_i)
    
    return dataset

In [ ]:
import inspect

def filter_kargs(target_function,**kargs):
    """Helper function to filter keyword arguments and only pass the needed ones in a function signature"""
    sig = inspect.signature(target_function)
    # check if there is a **kargs in the signature of the function, if yes it is ok as it will take care of the passed extra kargs
    if not any(p.kind == p.VAR_KEYWORD for p in sig.parameters.values()):
        extra_args = kargs.keys() - sig.parameters.keys()
        for args in extra_args:
            del kargs[args]
    return kargs

def debuf_karg_filter(target_function, **kargs):
    print("Kargs filtering:")
    print("before: {",*["{}:{}".format(key,val) for key, val in kargs.items()],"}")
    print("after: {",",".join(["{}:{}".format(key,val) for key, val in filter_kargs(target_function,**kargs).items()]),"}")

In [ ]:
import pandas as pd
filter_kargs(pd.read_csv, **{"sep":";", "truc":"test"})

### Manual entries

In [ ]:
dataset= GeologicalDataset()
print(dataset)

In [ ]:
dataset.head()

In [ ]:
dataset.get_observations()

In [ ]:
dataset.get_occurrence_observations()

In [ ]:
mogi.show_entity_qualities(dataset.get_observations(name="Dtest")[0])

In [ ]:
dataset.head()

In [ ]:
dataset.remove_observation_by_name("D8")
dataset.get_observations()

In [ ]:
dataset.remove_all_observations()
print(dataset)

In [ ]:
physical_space = PhysicalRepresentationSpace(coordinate_label= ["x","y","z"])
dataset.setup_representation_space(physical_space= physical_space)
dataset.add_occurrence_observation(name="DD", observed_object= "Keuper", occurrence= True, x= 1, z= 2, y= 0)
dataset.add_occurrence_observation(name="DN", observed_object= "Keuper", occurrence= False, x= 3, z= 2, y= 0)
dataset.update_extension()
print(dataset)

dataset.get_observations()

In [ ]:
dataset.head()

In [ ]:
dataset.add_orientation_observation(name="DO", observed_object= "Trias", dip= 30, dip_dir= 270, x= 2, z= 2, y= 0)
print(dataset)

dataset.get_observations()

In [ ]:
dataset.head()

### From file

In [ ]:
dataset = load_dataset_from_csv("../inputs/data_for_paper.csv", sep=";", index= "Id", coordinate_labels=["x","y","z"], labels={"ID":"Id", "strike":"dip_dir","name":"observed_object"})
print(dataset)

In [ ]:
dataset.head()

In [ ]:
# illustrates errors because of wrong separator
try:
    dataset2= load_dataset_from_csv("../inputs/data_for_paper.csv", labels={"ID":"Id", "strike":"dip_dir","name":"observed_object"})
except Exception as e:
    print(e)
    gkf = GeologicalKnowledgeManager().get_knowledge_framework()
    observations = gkf.search(type=gkf().Ponctual_Observation)
    gkf.remove_entities(observations)

In [ ]:
# illustrates errors when leaving wrong labels
try:
    dataset3 = load_dataset_from_csv("../inputs/data_for_paper.csv", sep=";", index= "ID")
except Exception as e:
    print(e)
    gkf = GeologicalKnowledgeManager().get_knowledge_framework()
    observations = gkf.search(type=gkf().Ponctual_Observation)
    gkf.remove_entities(observations)

## Creating a dataset

Data are actually described within the ontology, here thanks to the *Data* class.<br>
Adding new data points calls for creating new *Data* individuals (i.e., instances in the ontology).

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
data_head = np.array(['name', 'x', 'y', 'z', 'dip_dir', 'dip', 'observed_object','occurrence'])
data_array = np.array([['D1', 15, 20, 35, 270, 45, 'Trias_Base',True],
                       ['D2', 30, 25, 50, 270, 45, 'Trias_Base',True],
                       ['D3', 60, 30, 40, 90, 45, 'Trias_Base',True],
                       ['D4', 75, 15, 25, 90, 45, 'Trias_Base',True],
                       ['D5', 110, 20, 40, 270, 63, 'Trias_Base',True],
                       ['D6', 120, 20, 60, 270, 64, 'Trias_Base',True],
                       ['D7', 155, 20, 60, 89, 39, 'Trias_Base',True],
                       ['D8', 190, 20, 30, 91, 40, 'Trias_Base',True],
                       ['D11', 25, 22, 45, np.nan, np.nan, None ,True],
                       ['D22', 50, 22, 50, np.nan, np.nan, None,True],
                       ['D44', 100, 30, 20, np.nan, np.nan, None,True],
                       ['D77', 168, 30, 47, np.nan, np.nan, None,True]]
)
data_test = pd.DataFrame(data = data_array, columns = data_head)
data_test = data_test.astype({'name':str, 'x':float, 'y':float, 'z':float, 'dip_dir':float, 'dip':float, 'observed_object':str, 'occurrence':bool})
data_test.set_index("name", inplace = True)
data_test

In [ ]:
load_dataset_from_dataframe(data_test, coordinate_labels= ["x","y","z"])
print(dataset)

In [ ]:
dataset.head()

In [ ]:
from IPython.display import display, HTML

chart = HTML(dataset.info().replace("\n","<br>"))
display(chart)

chart = HTML(dataset.to_dataframe().to_html())
display(chart)


## Data visualisation

### Testing Data visualisation

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def draw_line(center, dip, dir, length= 1, ax= None, color = "black", **kargs):
    ax_plt = plt if ax is None else ax

    center = np.array(center)
    dip_rad = np.deg2rad(dip)
    vec_x =  np.cos(dip_rad)
    if dir == "left": vec_x *= -1
    vec_z = -np.sin(dip_rad)
    vect = 0.5 * length * np.array([vec_x,vec_z])
    start = center - vect
    end = center + vect
    ax_plt.plot([start[0],end[0]],[start[1],end[1]], color = color, **kargs)
    
    return vect
    
def draw_dip_symbol(center, dip, dir, length= 1, polarity= None, ax= None, color = "black", polarity_ratio= 0.4, **kargs):
    ax_plt = plt if ax is None else ax
    
    vect = draw_line(center= center, dip= dip, dir= dir, length= length, ax= ax_plt, color = color, **kargs)
    
    if polarity is not None:
        vect_pol = polarity_ratio * np.array([-vect[1],vect[0]])
        if (dir == "left" and polarity == "up") or (dir == "right" and polarity == "down") : vect_pol *= -1
        ax_plt.arrow(*center,*vect_pol, width=length/100, color = color, **kargs)
        

In [ ]:
draw_line([0,0],30, "left")
draw_dip_symbol([0,1],60, "right", polarity= "up", color= "red" )
plt.gca().set_aspect("equal")

### Object implementation

In [ ]:
import matplotlib.pyplot as plt
class Rendering(object):
    """Class dedicated to rendering a `RepresentationSpace`"""
    
    def __init__(self, space:RepresentationSpace):
        """Creating a rendering for the given `RepresentationSpace`"""
        self.space = space

class AxisAlignedCrossSection(Rendering):
    """A specialised `Rendering` that procudes a cross-section.
    
    The cross-section is defined by two coordinates of the rendered `PhysicalRepresentationSpace`"""
    
    def __init__(self, space:PhysicalRepresentationSpace, u= None, v= None, ax= None):
        """Creates a cross section through the given physical space
        
        Parameters:
        - space: the space through which the cross section is going.
        Note: the given space must be a `PhysicalRepresentationSpace` and have two or more coordinates
        - u: the label of the abscissa axis among the physical space coordinates.
        By default, if None is given, the first axis of the space is used.
        - v: the label of the ordinate axis among the physical space coordinates.
        By default, if None is given, the last axis of the space is used,
        effectively using both coordinates if the space is 2D, but a vertical cross-section if it is 3D.
        - ax: the matplotlib axis in which the space is to be rendered.
        If None (default), then a new axis will be created."""
        assert isinstance(space,PhysicalRepresentationSpace), "The given representation space must be a PhysicalRepresentationSpace, here: "+str(type(space))
        assert space.dimension > 1, "Cross sections are only possible through spaces of dimensions >=2, here: "+str(space.dimension)
        self.space = space
        
        if u is None:
            self.u = self.space.coordinate_labels[0]
            self.u_index = 0
        else:
            assert u in self.space.coordinate_labels, "The first given coordinate (u={}) is not in the represented space ({})".format(u, ",".join(space.coordinate_labels))
            self.u = u
            self.u_index = np.argwhere(self.space.coordinate_labels == self.u)[0,0]
        
        if v is None:
            self.v = space.coordinate_labels[-1]
            self.v_index = len(space.coordinate_labels) - 1
        else:
            assert v in space.coordinate_labels, "The second given coordinate (v={}) is not in the represented space ({})".format(v, ",".join(space.coordinate_labels))
            self.v = v
            self.v_index = np.argwhere(self.space.coordinate_labels == self.v)[0,0]
            
        self.plt_ax = None
        
    def setup_drawing(self, ax= None):
            if ax is not None:
                self.plt_ax = ax
            elif self.plt_ax is None:
                self.plt_ax = plt
            
            ax = self.plt_ax.gca() if self.plt_ax == plt else self.plt_ax
            ax.set_aspect("equal")
            ax.set_xlim( *self.space.extension[self.u_index])
            ax.set_xlabel(self.u)
            ax.set_ylim( *self.space.extension[self.v_index])
            ax.set_ylabel(self.v)
    
    def draw_line(self, center, dip, dir, length= 1, color = "black", setup_drawing= True, ax= None, **kargs):
        if setup_drawing: self.setup_drawing(ax)
        
        center = np.array(center)
        dip_rad = np.deg2rad(dip)
        vec_x =  np.cos(dip_rad)
        if dir == "left": vec_x *= -1
        vec_z = -np.sin(dip_rad)
        vect = 0.5 * length * np.array([vec_x,vec_z])
        start = center - vect
        end = center + vect
        self.plt_ax.plot([start[0],end[0]],[start[1],end[1]], color = color, **kargs)
        
        return vect        
    
    def draw_dip_symbol(self, center, dip, dir, length= None, polarity= None, color = "black", polarity_ratio= 0.4, setup_drawing= True, ax= None, **kargs):
        if setup_drawing: self.setup_drawing(ax)
        
        if length is None:
            length = np.abs(np.max(self.space.extension[:,1] - self.space.extension[:,0])) / 20
        
        vect = self.draw_line(center= center, dip= dip, dir= dir, length= length, color = color, setup_drawing=False, ax=ax, **kargs)
        
        if polarity is not None:
            vect_pol = polarity_ratio * np.array([-vect[1],vect[0]])
            if (dir == "left" and polarity == "up") or (dir == "right" and polarity == "down") : vect_pol *= -1
            self.plt_ax.arrow(*center,*vect_pol, width=length/100, color = color,  **kargs)
        
    def get_center(self,di):
        coord = self.space.get_object_coordinates(di)
        return coord[[self.u_index,self.v_index]]
    
    def draw_dip_data(self, ax= None, polarity= "up", setup_drawing= True, **kargs):
        if setup_drawing: self.setup_drawing(ax)
        
        for dataset_i in self.space.get_datasets():
            for di in dataset_i.get_orientation_observations():
                dir = "right" if di.dip_dir[0] < 180 else "left"
                center = self.get_center(di)
                self.draw_dip_symbol(center, di.dip[0], dir, polarity= polarity, setup_drawing= False, ax=ax, **kargs) 
    
    def draw_point(self, center, color = "black", marker="*", edge_color= "black", zorder= 10, setup_drawing= True, ax= None, **kargs):
        if setup_drawing: self.setup_drawing(ax)
        
        center = np.array(center)
        self.plt_ax.scatter(*center, color = color, marker= marker, edgecolors= edge_color, zorder=zorder, **kargs)
        
    def draw_occurrence_symbol(self, center, color = "lightblue", marker="*", edge_color= "black", zorder= 10, setup_drawing= True, ax= None, **kargs):
        if setup_drawing: self.setup_drawing(ax)
        self.draw_point( center, color, marker, edge_color= edge_color, zorder=zorder, setup_drawing= False, ax=ax, **kargs) 
        
    def draw_occurrence_data(self, ax= None, polarity= "up", setup_drawing= True, **kargs):
        if setup_drawing: self.setup_drawing(ax)
        
        for dataset_i in self.space.get_datasets():
            for di in dataset_i.get_occurrence_observations():
                center = self.get_center(di)
                self.draw_occurrence_symbol(center, setup_drawing= False, ax=ax, **kargs) 
    
    def show(self, ax= None, setup_drawing= True, **kargs):
        if setup_drawing: self.setup_drawing(ax)
        self.draw_occurrence_data(ax=ax, setup_drawing= False, **kargs)
        self.draw_dip_data(ax=ax, setup_drawing= False, **kargs)

In [ ]:
physical_space = PhysicalRepresentationSpace(coordinate_label=["x","y","z"])
dataset= GeologicalDataset(physical_space=physical_space)
print(dataset) 

In [ ]:
section = AxisAlignedCrossSection(physical_space)
section.draw_dip_symbol([1,30],30, "right", polarity= "up", length=10 )
section.draw_occurrence_symbol([100,40])

In [ ]:
section = AxisAlignedCrossSection(physical_space)
section.show()

### Tasks

The tasks in the algorithm of interpretation are not directly specified in the interpretation process but are made abstract to make it easier to implement alternative ways to proceed.

Our implementation separates two aspects of the problem:
1. **Task**: providing an interface for describing the task to be achieved
2. **Strategy**: providing the implementation for achieving the task.

Both are represented by a separate interface; only derived classes of Task that have a defined strategy are actually applicable.

There is a weak distinction between two kind of tasks:
- Strategies: for implementing parts of the overall algorithm that are fully dependent on the user's choice. In other terms, 
there is no obvious or ultimately best strategy for achieving a task, just actions that can be carried out in different ways.
- Heuristics: for providing quick and/or easy approximate solutions to a given problem. Here, there is in theory a correct answer
but it might be to difficult or expansive to infer it



In [ ]:

import sys
from enum import Enum, Flag, auto
import random, string

class MalissiaBaseError(Exception):
    """This is the base class for all exceptions of this package"""
    
class MissingTaskImplementationError(MalissiaBaseError):
    """Exception generated when trying to run a task for which no implementation was selected."""
    
    def __init__(self):
        super().__init__ ("This error occurred because a task has been executed without a proper implementation defined by a strategy.\n"\
            "Make sure to use fully implemented Task class (ie. not the base ones) and that self.strategy is set")
        

class StrategyType(Flag):
    """Defines flags for specifying behaviour of the Strategy.
    
    DEFAULT: no specific behaviour
    USER: it will ask user for some input
    RANDOM: it sill set some results in a random way
    BRUTE_FORCE: it will try all options and select the best result"""
    DEFAULT = 0
    USER = auto()
    RANDOM = auto()
    BRUTE_FORCE = auto()

class Strategy(object):
    """Strategies are pieces of algorithm that are implementing a specific task.
    
    Strategy provides the generic interface of algorithms implementing a `Task`
    and determines which tasks are actually concrete. In other words, a `Task` that does not inherit from `Strategy`
    can not be executed.
    
    `Strategy` provides the following interface (which has to be implemented in inheriting classes):
    - class attributes:
      - name: the human-readable name of the strategy
      - short_desc: a one-line description of the strategy
      - full_desc: a complete description of the strategy
    - attributes initialised by __init__:
      - strategy_type: the type of strategy (cf. `StrategyType`)
    - methods:
      - check_applicability(): which checks if the algorithm can be applied in the context defined by the task
      - execute(): which effectively runs the algorithm
    
    Strategies differ from Heuristics, in that the implemented task does not have an obvious/natural/true result or way to proceed,
    but it rather a matter of choice, hence a strategy."""
    
    name = "Strategy"
    short_desc = "Generic Abstract Strategy"
    full_desc = "This is not supposed to be implemented."
    strategy_type= StrategyType.DEFAULT
    
    def __init__(self, type= None, **kargs):
        """Initialises common attributes for the `Strategy` class
        
        Parameters:
        - type (`StrategyType`): the type of strategy implement, mostly to specify if user defined or random"""
        if type is not None:
            self.strategy_type = type
        
    def check_applicability(task = None):
        """Checks if this algorithm can be applied in the current context
        
        This base class method actually triggers an Exception to make sure the methods is implemented in inheriting class"""
        raise MalissiaBaseError("Trying to check applicability of the {} strategy but this class doesn't implement "\
            "the check_applicability method or run Strategy.check_applicability() instead.".format(__class__.__name__))
        
    def execute(self, task):
        """runs the algorithm
        
        This base class method actually triggers an Exception to make sure the methods is implemented in inheriting class.
        
        Parameters:
        - task: the task for which this strategy must be applied"""
        raise MalissiaBaseError("Trying to execute the {} strategy but this class doesn't implement the execute method "\
            "or run Strategy.execute() instead.".format(self.__class__.__name__))

class UserInputStrategy(Strategy):
    """Strategy based on interaction with the user."""
    name = "UserInputStrategy"
    short_desc = "Generic User Strategy"
    full_desc = "This is implementing generic user interaction."
    strategy_type= StrategyType.USER
    
    def __init__(self, **kargs):
        """Initialises a User based strategy"""
        super().__init__(**kargs)
        
    def check_applicability(task):
        """Checks if user interaction is possible (based on sys.ps1)
        
        This kind of algorithm is also restricted to result tasks because actions would be too complex to ask to users.
        """
        applicable = isinstance(task, ResultTask) 
        applicable &= UserInputStrategy.check_user_interface()
        return applicable
    
    def check_user_interface():
        return hasattr(sys, "ps1")
    
    def execute(self, task):
        """Asks user for the result to give.
        
        Parameters:
        - task: the task for which this strategy must be applied"""
        task.result = task.result_type(input("Please give in an input that can be used as a '{}':".format(task.result_type.__name__)))
    
class RandomStrategy(Strategy):
    """Strategy based on random generator."""
    name = "RandomStrategy"
    short_desc = "Generic Random Strategy"
    full_desc = "This is implementing generic random generator."
    strategy_type= StrategyType.RANDOM
    
    supported_types = (str, float, int)
    
    def __init__(self, **kargs):
        """Initialises a Random based strategy"""
        super().__init__(**kargs)
        
    def check_applicability(task):
        """Checks if applicable
        
        i.e., for a ResultTask and result_type being in supported_types
        """
        applicable = isinstance(task, ResultTask)
        applicable &= task.result_type in __class__.supported_types
        return applicable
    
    def execute(self, task):
        """apply the strategy.
        
        Parameters:
        - task: the task for which this strategy must be applied"""
        if task.result_type == str:
            task.result = "".join(random.choices(string.ascii_lowercase, k=10))
        elif task.result_type == float:
            task.result = random.random()
        elif task.result_type == int:
            task.result = random.randint(0,10)
        else:
            raise MalissiaBaseError("Unsupported type for RandomStrategy. Request: {}, Available: {}.".format(task.result_type,", ".join(__class__.supported_types)))
        
class Task(object):
    """Task is an abstract base class to provide the generic interface for any tasks of the `GeologicalInterpretationProcess`
    
    Task provides the following interface (which has to be implemented in inheriting classes):
    - class attributes:
      - name: the human-readable name of the task
      - short_desc: a one-line description of the task
      - full_desc: a complete description of the task
      - available_strategies: a list of available strategy types for implementing the `Task`.
        NB: strategies are related to `Task` by composition to make it possible to derive the tasks and still inherit from the base class strategies.
    - attributes initialised by __init__:
      - context: a context in which the task must be executed, typically the `GeologicalInterpretationProcess` instance
      - strategy: the selected strategy instance for implementing the task if any.
    """
    
    name = "Task"
    short_desc = "Generic Abstract Task"
    full_desc = "This is not supposed to be implemented."
    available_strategies = set()
    
    def __init__(self, context, **kargs):
        """Initialises common attributes for the `Task` class
        
        Parameters:
        - task_name: the human-readable name of the task
        - task_short_desc: a one-line description of the task
        - task_full_desc: a complete description of the task
        - context: the context of the task, this should provide the adapted interface for giving the required information
        """
        self.context = context

    def set_strategy(self, strategy):
        """Setter for the strategy, this ensures everything is set properly"""
        assert not ( (strategy is not None) and (not isinstance(strategy, Strategy))), "strategy must be either None or an instance of a Strategy (not a class)"
        self.strategy = strategy
        
    def check_applicability(self):
        """Checks that the task can be executed (ie. a strategy is set and applicable)"""
        if self.strategy is None: return False
        return self.strategy.__class__.check_applicability(self)
    
    def execute(self, bypass_applicability_check = False):
        """Method to run the selected implementation of the task.
        
        If no strategy is implemented/selected, this triggers an exception (`MissingTaskImplementationError`).
        
        parameters:
        - bypass_applicability_check (bool): if True, the applicability of the task won't be checked beforehand. Default: False"""
        if (not hasattr(self, "strategy")) or (self.strategy is None):
            raise MissingTaskImplementationError()
        elif (not bypass_applicability_check) and (not self.check_applicability()):
            raise MalissiaBaseError("executed a Task with a strategy that was not applicable in the current context.")
        else: self.strategy.execute(self)
        
class ActionTask(Task): 
    """Task in charge of applying some actions.
    
    Such tasks do not typically store a result but instead interact and possibly modify the context.
    """
    name = "ActionTask"
    short_desc = "Generic Abstract Task for actions"
    full_desc = "This is a generic action, meaning it doesn't hold a result but instead affects the context directly. This is not supposed to be implemented directly."
    available_strategies = Task.available_strategies
    
    def __init__(self, context, **kargs):
        """Initialises an ActionTask
        
        Parameter:
        - context: provides a context with the appropriate (task specific) interface. This parameter is compulsory."""
        super().__init__(context= context, **kargs)
        
class ResultTask(Task):
    """`Task` in charge of creating a result.
    
    Such tasks should not modify the context but only generate a result stored the `self.result` attribute.
    
    `UserInputStrategy` and `RandomStrategy` are automaticall added as available strategies
    """
    name = "ResultTask"
    short_desc = "Generic Task for generating results"
    full_desc = "This is a generic result generator, meaning it doesn't affect the context but holds a result instead."
    available_strategies = Task.available_strategies | set((UserInputStrategy, RandomStrategy))
    
    def __init__(self, result_type, context = None, **kargs):
        """Initialises a ResultTask
                
        Additional parameters (for others see `Task`):
        - result_type: the type of expeted result
        - context: provides a context with the appropriate (task specific) interface. This parameter is compulsory."""
        super().__init__(context= context, **kargs)
        self.result_type = result_type
        self.result = None

    def execute(self, bypass_applicability_check = False, return_result= True):
        """Additional behaviour for `ResultTask`
        
        Runs the `Task.execute()` then return result if the option is activated (default)"""
        super().execute(bypass_applicability_check= bypass_applicability_check)
        if return_result: return self.result
        
class StrategyFactory(object):
    """This class gathers all the strategies available for the `GeologicalInterpretationProcess`.
    
    The strategies are automatically recovered from the subclasses of Task and Strategy.
    Three dictionaries are registering the task and corresponding strategies:
    - self.tasks: are the Defined Tasks having defined possible Strategies
    - self.specified_tasks: are the tasks with a specific available strategy (i.e., only one)
    - self.implemented_tasks: tasks that have an implemented strategy
    NB: they aren't used for generating the tasks, but just for reporting the existing tasks.
    """
    
    def __init__(self):
        """Initialisation of the strategy storage."""
        self.tasks = {}
        self.specified_tasks = {}
        self.implemented_tasks = {}
        self.tasks_without_strategy = {}
        self.list_available_tasks()
        self.list_specified_tasks()
        self.list_implemented_tasks()
        self.list_tasks_without_strategy()
        
    def generate_task(self, task_type, strategy_type= None, strategy_class= None, context= None, **kargs):
        """Creates a task of a given type and instanciate appropriate strategy
        
        Parameters:
        - task_type: the class of the task to be created
        - strategy_type: the type of strategy to be used, see `StrategyType`
        - strategy_class: to force the strategy selection, its class can be directly given here
        - context: a context that can be passed to the task to define it (typically the `GeologicalInterpretationProcess`)
        - kargs: keyword arguments that can be passed to the task initialisation"""
        
        # Create the task
        task = task_type(context = context, **kargs)
        
        if strategy_class is None:
            # list strategies
            strategies = np.array(list(task_type.available_strategies))
            
            # check applicability to the task and context
            applicable_strategy = [candidate_strategy for candidate_strategy in strategies if candidate_strategy.check_applicability(task)]
            
            # filter strategy types
            if strategy_type is not None:
                applicable_strategy = [candidate_strategy for candidate_strategy in applicable_strategy if candidate_strategy.strategy_type == strategy_type]
                
            if len(applicable_strategy) == 0:
                message = ["No implementation of the requested task ({}) found in this context.".format(task_type)]
                message += ["Note that the following strategies were available:\n"+"\n".join(strategies)]
                message += ["Note that the following strategies were applicable:\n"+"\n".join(applicable_strategy)]
                message += ["Note that the following strategies were filtered out:\n"+"\n".join([candidate_strategy for candidate_strategy in applicable_strategy if candidate_strategy.strategy_type != strategy_type])]
                raise MalissiaBaseError("\n".join(message))
             
            # this could be defined by a strategy as well, e.g. pick first of random
            strategy_class = random.choice(applicable_strategy)
            
        else:
            if strategy_class.check_applicability(task) == False:
                raise MalissiaBaseError("The proposed implementation ({}) of the requested task ({}) is not applicable in this context.".format(strategy_class,task_type))
                
        # instanciate the strategy
        strategy_instance = strategy_class(**kargs)
        task.set_strategy(strategy_instance)
            
        return task
    
    def list_available_tasks(self):
        """List the Tasks that are fully defined (ie. having attached strategies)"""
        task_pile = list(Task.__subclasses__())
        while len(task_pile) > 0:
            task_i = task_pile.pop()
            
            strategies = task_i.available_strategies
            if len(strategies) > 0:
                self.tasks[task_i] = strategies 
            
            sub_tasks = list(task_i.__subclasses__())
            task_pile += sub_tasks
    
    def list_specified_tasks(self):
        """List the Tasks that have only one possible strategy"""
        task_pile = list(Task.__subclasses__())
        while len(task_pile) > 0:
            task_i = task_pile.pop()
            
            strategies = task_i.available_strategies
            if len(strategies) == 1:
                self.specified_tasks[task_i] = list(strategies)[0]
            
            sub_tasks = list(task_i.__subclasses__())
            task_pile += sub_tasks
    
    def list_implemented_tasks(self):
        """List the Tasks that are implemented (ie. having an attached strategy instance)"""
        task_pile = list(Task.__subclasses__())
        while len(task_pile) > 0:
            task_i = task_pile.pop()
            
            if hasattr(task_i,"strategy") and task_i.strategy is not None:
                self.implemented_tasks[task_i] = task_i.strategy 
            
            sub_tasks = list(task_i.__subclasses__())
            task_pile += sub_tasks
            
    def list_tasks_without_strategy(self):
        """Lists the tasks that have no strategies nor subclass"""
        task_pile = list(Task.__subclasses__())
        while len(task_pile) > 0:
            task_i = task_pile.pop()
            
            strategies = task_i.available_strategies
            if (len(strategies) == 0) and (len(task_i.__subclasses__()) == 0):
                self.tasks_without_strategy[task_i] = ()
            
            sub_tasks = list(task_i.__subclasses__())
            task_pile += sub_tasks
    
    def __str__(self):
        """Describe the stored strategies"""
        self.list_available_tasks()
        self.list_specified_tasks()
        self.list_implemented_tasks()
        self.list_tasks_without_strategy()
        
        desc = ["Strategy Factory:"]
        desc+= ["- Available Tasks:"]
        for task, strat_list in self.tasks.items():
            desc += ["  |- {}:".format(task.name)]
            if len(strat_list) == 0:
                desc += ["  |[Warning]: no strategy provided for this task"] 
            for strat_i in strat_list:
                desc += ["  |  |- {}{}: {}".format(strat_i.__class__.__name__, "" if strat_i.strategy_type is StrategyType.DEFAULT else "["+strat_i.strategy_type.name+"]", strat_i.short_desc)]
        
        if len(self.specified_tasks) > 0:
            desc+= ["- Specified Tasks:"]
            for task, strat in self.specified_tasks.items():
                desc += ["  |- {}: {}{}({})".format(task.name, strat.name, "" if strat.strategy_type is StrategyType.DEFAULT else "["+strat.strategy_type.name+"]", strat.short_desc)]
        if len(self.implemented_tasks) > 0:
            desc+= ["- Implemented Tasks:"]
            for task, strat in self.implemented_tasks.items():
                desc += ["  |- {}: {}{}({})".format(task.name, strat.name, "" if strat.strategy_type is StrategyType.DEFAULT else "["+strat.strategy_type.name+"]", strat.short_desc)]
        if len(self.tasks_without_strategy) > 0:
            desc+= ["- Tasks without strategy:"]
            for task, strat in self.tasks_without_strategy.items():
                desc += ["  |- {}: [Warning]: no strategy provided for this task".format(task.name)]
        return "\n".join(desc)
        
    
__default_strategy_factory__ = StrategyFactory()
    
class HeuristicsFactory(object):
    """This class provides Heuristics for the `GeologicalInterpretationProcess`.
    
    Heuristics are pieces of algorithm that are implementing a specific computational or decisionnal task.
    They differ from strategies, in that there exists a true result even if it is not know or two complex to infer accurately.
    Heuristics try to provide acceptable approximate results with a simple algorithms to save either computation time, memory, or complexity.

    The heuristics are stored in a dictionnary (`self.__heuristics`) that associates tasks with a list of possible implementation. 
    """
    
    def __init__(self):
        """Initialises the Heuristics storage."""
        self.__heuristics = {}
        
print(__default_strategy_factory__)

In [ ]:
class ExampleTask(ResultTask):
    """class in charge of showing how to implement new tasks
    
    Such task is independent of a context and ignores it if passed."""
    
    name = "ExampleTask"
    short_desc = "gives an example"
    full_desc = "Demonstrates the implementation of new tasks."
    def __init__(self, result_type= None, **kargs):
        """example of initialisation of a result task
        
        Parameters:
        - result_type: the type of expeted result. If None, one of the RandomStrategy supported type is used."""
        result_type = result_type if result_type is not None else random.choice(RandomStrategy.supported_types)
        super().__init__(result_type= result_type)
        
class UserExampleTask(ExampleTask):

    name = "UserExampleTask"
    short_desc = "example user strategy"
    full_desc = "Gives an example of implementation of user defined algorithm."
    available_strategies = {UserInputStrategy}
    
    def __init__(self, result_type= str, **kargs):
        super().__init__(result_type= result_type)
        strategy = UserInputStrategy()
        super().set_strategy(strategy)
    
class RandomExampleTask(ExampleTask):

    name = "RandomExampleTask"
    short_desc = "example random strategy"
    full_desc = "Gives an example of implementation of random defined algorithm."
    available_strategies = {RandomStrategy}
    
    def __init__(self, result_type= str, **kargs):
        super().__init__(result_type= result_type)
        strategy = RandomStrategy()
        super().set_strategy(strategy)
        
print(__default_strategy_factory__)


In [ ]:
task = __default_strategy_factory__.generate_task(ExampleTask, strategy_type= StrategyType.RANDOM)
task.execute()
if isinstance(task,ResultTask):
    print(task.result)

In [ ]:
task = __default_strategy_factory__.generate_task(ExampleTask)
task.execute()
if isinstance(task,ResultTask):
    print(task.result)

In [ ]:
try:
    strat = Strategy()
    strat.check_applicability()
except MalissiaBaseError as e:
    print(e)

In [ ]:
task = RandomExampleTask()
if task.check_applicability():
    task.execute()
task.result

In [ ]:
task = ActionTask(None)
task.name

In [ ]:
try:
    task.execute()
except MalissiaBaseError as e:
    print(e)

In [ ]:
ExampleTask.available_strategies

In [ ]:
UserExampleTask.available_strategies

In [ ]:
ActionTask.available_strategies

In [ ]:
task = UserExampleTask()
if task.check_applicability():
    task.execute()
task.result

### Specific Tasks implementation

In [ ]:
class UserSelectStrategy(UserInputStrategy):
    """Strategy based on interaction with the user."""
    name = "UserSelectStrategy"
    short_desc = "Selection User Strategy"
    full_desc = "This is implementing selection based on user interaction."
    type= StrategyType.USER
    
    def __init__(self, **kargs):
        """Initialises a User based strategy"""
        super().__init__(**kargs)
        
    def check_applicability(task, context = None):
        """Checks if applicable
        
        uses UserInputStrategy.check_applicability and SelectionTask type
        """
        applicable = isinstance(task, SelectionTask) 
        applicable &= UserInputStrategy.check_user_interface()
        return applicable
    
    def execute(self, task):
        """Asks user for the result to give.
        
        Parameters:
        - task: the task for which this strategy must be applied"""
        message = "Please select an option amongst the following (give its index):\n "+"\n".join(["{}: {}".format(i,val) for i, val in enumerate(task.choices)])
        try:
            selected = int(input(message))
            task.result = task.result_type(task.choices[selected])
        except ValueError:
            raise MalissiaBaseError("User input must be in the form of an integer corresponding to the selected index.")
        except IndexError:
            raise MalissiaBaseError("The selected index must be within [{}, {}].".format(0,len(task.choices)-1))
        return task.result
    
class RandomSelectStrategy(RandomStrategy):
    """Strategy based on random generator."""
    name = "RandomSelectStrategy"
    short_desc = "Random Selection Strategy"
    full_desc = "This is implementing a random selection."
    type= StrategyType.RANDOM
    
    def __init__(self, **kargs):
        """Initialises a Random based strategy"""
        super().__init__(**kargs)
        
    def check_applicability(task, context = None):
        """Checks if applicable
        
        Needs to be set for a SelectionTask with at least one choice
        """
        applicable = isinstance(task, SelectionTask)
        applicable &= len(task.choices) >0
        return applicable
    
    def execute(self, task):
        """perform random selection
        
        Parameters:
        - task: the task for which this strategy must be applied"""
        
        if len(task.n) == 1:
            if task.n[0] == 1:
                task.result = random.choice(task.choices)
                return task.result_type(task.result)
            else:
                k = task.n
        else:
            k = np.cumprod(task.n)[-1]
            
        task.result = random.choices(task.choices, k = k)
        if len(task.n) > 1:
            task.result.reshape(task.n)
            
        return task.result.astype(task.result_type)

class SelectionTask(ResultTask):
    """SelectionTask performs a selection among a series of possible results."""
    
    name = "SelectionTask"
    short_desc = "makes a selection"
    full_desc = "Performs a selection among a series of possible results."
    available_strategies = set((UserSelectStrategy, RandomSelectStrategy))
    
    def __init__(self, choices, n= 1, result_type= None, context= None, **kargs):
        """Initialisation of a selection task
        
        Parameters:
        - choices: a series of possible values for the result. Will be transformed into a numpy array and flatten (np.array(choices).ravel())
        - n (int): number of elements to be picked, alternatively if a shape is given, the shape of the expected result array
        - result_type: the type of expeted result. If None, the type of the choices will be used by default."""
        self.choices = np.array(choices).ravel()
        self.n = np.array(n).ravel()
        result_type = result_type if result_type is not None else self.choices.dtype.type
        super().__init__(result_type= result_type)
        


In [ ]:
task = __default_strategy_factory__.generate_task(SelectionTask, choices = [1,2,27,42])
task.execute()
if isinstance(task,ResultTask):
    print(task.result)

In [ ]:
choices = dataset.get_observations()
task = __default_strategy_factory__.generate_task(SelectionTask, choices = choices, strategy_type= StrategyType.RANDOM)
task.execute()
if isinstance(task,ResultTask):
    print(task.result)
print("Result type:",type(task.result))

In [ ]:
class SelectObservation(SelectionTask):
    """Selection of an observation"""
    name = "SelectObservation"
    short_desc = "selects observations"
    full_desc = "Performs a selection among possible observations."
    
    def __init__(self, n= 1, choices = None, dataset= None, context= None, **kargs):
        """Initialisation of a selection task
        
        Parameters:
        - choices: a series of possible observations. If None (default), the dataset in the context will be used (and therefore can't be None)
        - dataset: a GeologicalDataset to bring the available observations unless already defined by choices 
        - context: if given a `GeologicalInterpretationProcess`, will be used to define the possible selection unless, choices or dataset are given.
        - n (int): number of elements to be picked, alternatively if a shape is given, the shape of the expected result array"""
        choices = choices if choices is not None else self.get_choices(dataset= dataset, context= context, **kargs)
        super().__init__(choices = choices, n= n, context = context, **kargs)

    def get_choices(self, choices = None, dataset:GeologicalDataset= None, context= None, **kargs):
        """generates the choices list from different possible sources
        
        Parameters:
        - choices: a series of possible observations. If None (default), the dataset in the context will be used (and therefore can't be None)
        - dataset: a GeologicalDataset to bring the available observations unless already defined by choices 
        - context: if given a `GeologicalInterpretationProcess`, will be used to define the possible selection unless, choices or dataset are given.
        """
        if (context is None) and (dataset is None):
            raise MalissiaBaseError("Either a series of choices, a dataset, or a context must be given.")
        if dataset is None:
            dataset = context.dataset
        choices = dataset.get_observations()
        return choices

In [ ]:
SelectObservation.available_strategies

In [ ]:
task = __default_strategy_factory__.generate_task(SelectObservation, dataset = dataset)
task.execute()
if isinstance(task,ResultTask):
    print(task.result)
print("Result type:",type(task.result))

## Interpretation Workflow

The interpretation process in itself is run in a **GeologicalInterpretationProcess** and follow a very simple and generic algorithm.<br>
This algorithm implements a Deming wheel process of continual improvement:
1. Plan:
    1. Select a situation
    2. Select an action
2. Do: Implement the action (e.g., CreateInterpretationElement)
    1. List features
    2. Identify possible explanations
    3. Rank/chose explanations
    4. Instanciate individuals
    5. Infer and set parameters
3. Check: Evaluate consistency
    1. Evaluate internal consistency
    2. Evaluate relational likelihood
    3. Evaluate feature explanation
4. Act: Generate anomalies and report

In [ ]:
class TerminationFlag(Flag):
    """Defines flags for specifying why the `GeologicalInterpretationProcess` stoped
    
    DEFAULT: it has not been terminated
    USER: was terminated by user
    MAX_ITER: maximum iteration number reached
    """
    DEFAULT = 0
    USER = auto()
    MAX_ITER = auto()
    
class GeologicalInterpretationProcess(object):
   """GeologicalInterpretationProcess implements the core process of a geological intepretation.
   
   It connects all the required elements and resulting artefacts relatively to a given interpretation sequence:
    - knowledge_framework: a GeologicalKnowledgeFramework
    - dataset: a GeologicalDataSet that interfaces all the vailable data
    - strategies: a StrategyFactory that implements various way to perform required tasks
    - heuristics: a HeuristicFactory that provides various algorithms for computing or inferring parameters"""
     
   def __init__(self,
                dataset: GeologicalDataset,
                physical_space: PhysicalRepresentationSpace = None,
                knowledge_framework= None,
                strategies: StrategyFactory= None):
      """Creates a GeologicalInterpretationProcess
        
      ---------------------------
      Parameters:
       - dataset (GeologicalDataset): a dataset to be explained by this interpretor
       - physical_space (PhysicalRepresentationSpace): the physical space that must be explainined by the interpretation.
       If None (default), the space of the dataset is used instead.
       - knowledge_framework: a GeologicalKnowledgeFramework that defines the concepts used for this interpretation.
         If None is given, the the default knowledge framework is used (`GeologicalKnowledgeManager().get_knowledge_framework()`)
       - strategies: pointer to the factory to be used for accessing strategies for the various tasks in this process
         If None (default), the default factory is used as defined by __default_strategy_factory__
      """
      self.dataset = dataset
      self.physical_space = physical_space if physical_space is not None else self.dataset.physical_space
      self.strategies = strategies if strategies is not None else __default_strategy_factory__
      self.knowledge_framework= GeologicalKnowledgeManager().get_knowledge_framework() if knowledge_framework is None else knowledge_framework
      
      self.update_status()
   
   def update_status(self, init= False):
      """Performs status update operations
      
      If the status is not set yet, this method will also create it and initialise it.
      
      Parameters:
      - init (Bool): if True, the status will be reset to initial values. Default is False."""
      
      if not hasattr(self, "status"):
         self.status = {}
         init = True
      if init:
         self.status["epoch"] = None # index of the current iteration
      
      self.status["coverage"] = self.evaluate_coverage() # ratio of representation space covered by an explanation
      self.status["data_explanation_ratio"] = self.evaluate_data() # ratio of explained observations
      self.status["anomaly_explanation_ratio"] =  self.evaluate_anomalies() # ratio of explained observations
      self.status["termination_criterion"] = TerminationFlag.DEFAULT # gives explanation about why it terminated
      
   def evaluate_coverage(self):
      """Evaluates the amount of physical space that is explained
      
      Returns:
      - a float between 0 and 1, representing the ratio of covered space"""
      return 0
      
   def evaluate_data(self):
      """Evaluates the amount of data that is explained
      
      Returns:
      - a float between 0 and 1, representing the ratio of explained data"""
      return 0
   
   def evaluate_anomalies(self):
      """Evaluates the amount of raised anomalies that are now explained
      
      Returns:
      - a float between 0 and 1, representing the ratio of explained anomalies"""
      return 0
   
   def __str__(self):
      """Return a report describing the interpretation process"""
      desc = ["Geological Interpretation Process:"]
      if self.dataset is None:
         desc+= ["|- Dataset: None"]
      else:
         desc+= ["|- Dataset: "+"\n| |".join(self.dataset.__str__().split("\n"))]
         
      if self.knowledge_framework is None:
         desc+= ["|- Geological Knowledge Framework: None"]
      else:
         desc+= ["|- "+"\n| |".join(self.knowledge_framework.__str__().split("\n"))]
         
      if self.strategies is None:
         desc+= ["|- Tasks with strategies: None"]
      else:
         desc+= ["|- "+"\n| |".join(self.strategies.__str__().split("\n"))]
         
      return "\n".join(desc)
   
   def run(self, max_iter= None):
      """Runs the iterative interpretation process
      
      Parameters:
      - max_iter (int): if set, it specifies the maximum number of iterations to run before terminating.
      Iteration (epoch) are numbered starting at 0, this way the counter also represents the number of passed iterations.
      """
      
      # run the iterative process as long as a termination criterion is not reached
      
      #first update the status to make sure it is up to date, and reset the termination criteria
      self.update_status()
      
      # new epoch increments the iteration count and check termination
      try:
         while self.new_epoch(max_iter = max_iter):
         
            # 1. Plan
            # -----------------------------
            # 1.1 chose a type of action
            
            # 1.2 select a situation (ie. features to be explained in a context)
            situation = self.strategies.generate_task(SelectSituation, context = self).execute(return_result= True)
            print("{}: {}".format(self.status["epoch"],situation))
            
            # 2. Do
            # -----------------------------
            # 2.1 execute the specified action
            
            # 3. Check
            # -----------------------------
            #  3.1 Evaluate internal consistency
            #  3.2 Evaluate relational likelihood
            #  3.3 Evaluate feature explanation
            
            # 4. Act
            # -----------------------------
            # 4.1 Generate anomalies and report
            # 4.2 update status
            
      except KeyboardInterrupt:
         self.status["termination_criterion"] = TerminationFlag.USER
         
      return self.status["termination_criterion"]
         
   def new_epoch(self, max_iter= None):
      """Starts a new epoch (iteration) unless termination criteria were reached
      
      Returns:
      - True if the new epoch should start, False if iterations should stop
      - max_iter (int): if set, it specifies the maximum number of iterations to run before terminating"""
      keep_going = True
      
      self.increment_epoch()
      if (max_iter is not None) and (self.status["epoch"] >= max_iter):
         self.status["termination_criterion"] = TerminationFlag.MAX_ITER
         return False
      
      return keep_going
      
   def increment_epoch(self):
      """Initialize or increment the epoch count"""
      if self.status["epoch"] is None:
         self.status["epoch"] = 0
      else:
         self.status["epoch"] += 1
      
    

### Interpretation Process Tasks

In [ ]:
class InterpretationSituation(object):
    """Defines a situation of interpretation.
    
    A situation is gathering:
    - self.features: features to be explained
    - self.context: an interpretation context, ie. elements to be considered during the interpretation
    - self.process: the `GeologicalInterpretationProcess` in which this situation takes place"""
    
    def __init__(self, features, context, process:GeologicalInterpretationProcess) -> None:
        """Initialises the situation"""
        self.features = np.array(features).ravel()
        self.context = np.array(context).ravel()
        self.process = process

class RandomSituationStrategy(Strategy):
    """Strategy for randomly selecting a situation"""
    name = "RandomSituationStrategy"
    short_desc = "Random Situation Selection"
    full_desc = "This is implementing random selection of situation to be explained."
    strategy_type= StrategyType.RANDOM
    
    def __init__(self, **kargs):
        """Initialisation of radom situation selection"""
        super().__init__(**kargs)
        
    
    def check_applicability(task):
        """Checks if applicable
        
        i.e., for a SelectSituation
        """
        applicable = isinstance(task, SelectSituation)
        applicable &= task.context is not None
        return applicable
    
    def execute(self, task):
        """apply the strategy.
        
        Parameters:
        - task: the task for which this strategy must be applied"""
        
        feature_selection_task = task.context.strategies.generate_task(SelectObservation, strategy_type= StrategyType.RANDOM, context= task.context)
        selected_feature = feature_selection_task.execute(return_result= True)
        
        task.result = InterpretationSituation( selected_feature, context= [], process= task.context)
  
class SelectSituation(ResultTask):
    """Task for selecting a situation to be explained (feature to be explained + interpretation context)"""
    
    name = "SelectSituation"
    short_desc = "selects a situation to ne explained"
    full_desc = "Selects features to be explained and an interpretation context."
    result_type = InterpretationSituation
    available_strategies = set((RandomSituationStrategy,))
    
    def __init__(self, context:GeologicalInterpretationProcess, feature_choices = None, n= None, **kargs):
        """Initialisation of a situation selection task
        
        Parameters:
        - feature_choices: a preselected list of features to be explained. If None (default), the dataset in the context will be used 
        - context: a `GeologicalInterpretationProcess`, as this is an action task, this context will be modified by the task.
        It will also be used to define the possible selection unless feature_choices is given.
        - n (int): number of features to be selected, if None, this is inferred from the strategies in the context."""
        self.feature_choices = feature_choices
        self.n = n
        super().__init__(context = context, result_type= SelectSituation.result_type, **kargs)
        
    

In [ ]:
gip = GeologicalInterpretationProcess(dataset)
print(gip)

In [ ]:
task = __default_strategy_factory__.generate_task(SelectSituation, context = gip)
task.execute()
if isinstance(task,ResultTask):
    print(task.result)
print("Result type:",type(task.result))

In [ ]:
gip.run(max_iter=2)

In [ ]:
print(gip)